# 1. Librerias y módulos

### 1.1. Instalar librerias

In [1]:
# Extracción de datos
!pip install pdfminer.six
!pip install pdfquery
!pip install 'PyPDF2<3.0'
!pip install "camelot-py[base]"
!pip install ghostscript
!apt install ghostscript python3-tk
!pip install xlsxwriter

# Conversión a jpg
!pip install pdf2image
!sudo apt-get install python-poppler
!sudo apt-get install poppler-utils

# Conversion a txt
!sudo add-apt-repository ppa:alex-p/tesseract-ocr-devel -y
!sudo apt-get update
!sudo apt-get install -y tesseract-ocr
!sudo apt-get install tesseract-ocr-spa
!pip install pytesseract
!pip install pyyaml==5.1
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package python-poppler
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 174 kB of archives.
After this operation, 754 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.1 [174 kB]
Fetched 174 kB in 1s (309 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend:

### 1.2. Importar librerias

In [2]:
# Google Colab y Drive
from google.colab import drive
import sys

# Extracción de datos
import os
import io
import camelot
import ghostscript
import PyPDF2
import re
import xlsxwriter
from io import StringIO
import pandas as pd
import numpy as np
from pdfquery import PDFQuery
from pathlib import Path
import itertools
from itertools import chain
import matplotlib.pyplot as plt
from matplotlib import patches
import pdfminer
from pdfminer.high_level import extract_pages
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator 
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.layout import LTPage
from pdfminer.layout import LTTextBoxHorizontal
from pdfminer.layout import LTTextBoxVertical
from pdfminer.layout import LTTextLineHorizontal
from pdfminer.layout import LTTextLineVertical
from pdfminer.layout import LTTextContainer
from pdfminer.layout import LTChar
from pdfminer.layout import LTText
from pdfminer.layout import LTTextBox
from pdfminer.layout import LTAnno

# Conversion a jpg
from pdf2image import convert_from_path
from pathlib import Path
import PIL.Image
from glob import glob

# Conversion a txt
import json
import cv2
import random
import cv2
import pytesseract
from pytesseract import Output
import matplotlib.pyplot as plt
from pdf2image import pdfinfo_from_path

### 1.3. Importar el módulo "funciones"

In [ ]:
drive.mount( '/content/gdrive' )
!cat '/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/funciones.py'
sys.path.append( '/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento' )

In [4]:
import funciones as fun

# 2. Modalidad: Servicio de Control Específico a Hechos con Presunta Irregularidad

In [5]:
modalidad           = 'SEHPI'
modalidad_ext       = 'SERVICIO_DE_CONTROL_ESPECIFICO_A_HECHOS_CON_PRESUNTA_IRREGULARIDAD'

## 2.1. Extraer archivos resumen

In [ ]:
input_path      = glob( f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/datos/SERVICIO_CONTROL_POSTERIOR/{ modalidad_ext }/*/*-resumen.pdf' )
output_path     = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/1_archivos_resumen'
txt_output_path = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/6_archivos_registro'

txt_file_path = os.path.join( txt_output_path, f'{ modalidad }_resumen_info.txt.txt' )

with open( txt_file_path, 'w+' ) as file:
    for i, input_element in enumerate( input_path ):
        
        try:      
            input_element_name                    = input_element[ input_element.rindex( '/' ) + 1: ]
            input_element_name                    = os.path.splitext( input_element_name )[ 0 ]
            
            extracted_input_element               = fun.extract_all( input_element, modalidad )
            extracted_input_element[ 'doc_name' ] = input_element_name

            output_element_path = os.path.join( output_path, f'{ input_element_name }.xlsx' )

            extracted_input_element.to_excel( output_element_path )

            line = f'Success extracting: { output_element_path }\t{ input_element }\n'
            file.write( line )
            
        except Exception as e:
            error_line = f'Error extracting: { input_element }\n'
            file.write( error_line )

## 2.2. Hacer append de archivos resumen

In [10]:
input_path          = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/1_archivos_resumen/*.xlsx'
output_path         = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/5_archivos_concatenados'

input_elements      = glob( input_path )
tables              = []

for input_element in input_elements:
  
    table           = pd.read_excel( input_element, dtype = { 'dni': object } )
    table           = table.fillna( '.' )

    tables.append( table )

ac_summary_table    = pd.concat( tables, axis = 0 )
output_element_path = os.path.join( output_path, f'{ modalidad }_summary_table.xlsx' )

ac_summary_table.to_excel( output_element_path )

## 2.3. Extraer fechas

### 2.3.1. Convertir la primera página de los informes a formato jpg

In [ ]:
input_path      = glob( f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/datos/SERVICIO_CONTROL_POSTERIOR/{ modalidad_ext }/*/*-informe.pdf' )
output_path     = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/2_archivos_jpg'
txt_output_path = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/6_archivos_registro'

fun.convert_1p_to_jpg( input_path, output_path, txt_output_path, modalidad )

### 2.3.2. Convertir los archivos jpg a formato txt usando OCR

In [ ]:
input_path      = glob( f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/2_archivos_jpg/*-informe.jpg' )
output_path     = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/3_archivos_txt'
txt_output_path = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/6_archivos_registro'

fun.convert_1p_to_text( input_path, output_path, txt_output_path, modalidad )

## 2.3.3. Extraer las fechas usando regex

In [7]:
input_path      = glob( f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/3_archivos_txt/*-informe.txt' )
output_path     = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/4_archivos_dates_xlsx'
txt_output_path = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/6_archivos_registro'
regex           = r'(?P<day1>\d{1,2})\s?(De|de|DE|DE-|E|DE.)?(\:|-)?\s?(?P<month1>\w{4,10})?\s?(\—)?\s?(De|DE|de|E|e|ce|Es|DE-|DE/|DE.)?\s?(-)?\s?(?P<year1>\d{4})?\s?(Al|aL|AL|A|al|a|¿AL)\s?(?P<day2>\d{1,2})\s?(De|de|DE|DE-|E|DE.)?\s?(BS)?\s?(?P<month2>\w{4,10})\n?(.)?(/)?\s?(De|DE|de|E|e|ce|Es|DE-|DE/|DE.)?\n?(OS|nd|-)?\s?(?P<year2>\d{4})'

fun.extract_dates( input_path, output_path, txt_output_path, modalidad, regex )

## 2.3.4. Hacer un append vertical de las fechas extraidas

In [8]:
input_path   = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/4_archivos_dates_xlsx/*.xlsx'
output_path  = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/5_archivos_concatenados'

fun.append_dates( input_path, output_path, modalidad )

,numero_informe,inicio,final
0,011-2020-2-0421,01/01/2019,05/11/2020
1,032-2021-2-0393,23/08/2019,08/03/2021
2,027-2020-2-0361,02/01/2015,30/06/2016
3,001-2021-2-0362,01/03/2018,31/10/2018
4,013-2022-2-0405,08/11/2019,28/02/2020
...,...,...,...
1912,001-2022-2-2930,27/12/2019,28/02/2021
1913,002-2020-2-1762,26/09/2019,31/12/2019
1914,002-2021-2-0354,02/09/2019,13/03/2020
1915,002-2021-2-0392,13/02/2019,30/06/2020


# 2.4. Merge

In [11]:
sehpi_summary_table                     = pd.read_excel( f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/5_archivos_concatenados/{ modalidad }_summary_table.xlsx' )
sehpi_dates                             = pd.read_excel( f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/5_archivos_concatenados/{ modalidad }_fechas.xlsx' )

sehpi_summary_table[ 'numero_informe' ] = sehpi_summary_table[ 'numero_informe' ].astype( str ).str.strip().\
                                                                                          str.replace( '/', '_' ).\
                                                                                          str.replace( '\\', '_', regex = True )
sehpi_dates[ 'numero_informe' ]         = sehpi_dates[ 'numero_informe' ].astype( str ).str.strip()

sehpi_total = pd.merge( sehpi_summary_table, 
                        sehpi_dates, 
                        how = 'outer',
                        on  = 'numero_informe' )

sehpi_total.to_excel( f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/7_archivos_finales/{ modalidad }_total.xlsx' )

***FIN***